In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from numpy import concatenate
from math import sqrt
from statistics import mean
import random
import matplotlib.pyplot as plt

/Users/alexellard/opt/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('/Users/alexellard/Downloads/scenarios_data.csv', header=0, index_col=0)
df

,reservoir_demand,node_1_demand,node_2_demand,node_3_demand,node_4_demand,node_5_demand,node_6_demand,node_7_demand,node_8_demand,node_9_demand,...,link_23_flow,link_24_flow,link_25_flow,link_26_flow,link_27_flow,link_28_flow,link_29_flow,link_30_flow,link_31_flow,link_32_flow
Timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,-3348.0,154.8,169.2,18.0,118.8,169.2,234.0,86.4,82.8,104.4,...,439.2,-205.2,-75.6,-18.0,118.8,68.4,7.2,-54.0,75.6,212.4
2017-01-01 00:30:00,-2959.2,136.8,147.6,18.0,108.0,144.0,237.6,79.2,75.6,90.0,...,392.4,-187.2,-72.0,-21.6,108.0,61.2,3.6,-54.0,72.0,183.6
2017-01-01 01:00:00,-2692.8,129.6,126.0,14.4,104.4,133.2,205.2,72.0,72.0,90.0,...,349.2,-176.4,-64.8,-18.0,93.6,50.4,3.6,-39.6,54.0,151.2
2017-01-01 01:30:00,-2379.6,104.4,100.8,10.8,86.4,122.4,180.0,68.4,57.6,82.8,...,316.8,-158.4,-54.0,-10.8,79.2,46.8,7.2,-32.4,50.4,140.4
2017-01-01 02:00:00,-2228.4,104.4,104.4,10.8,79.2,122.4,183.6,68.4,54.0,75.6,...,277.2,-144.0,-57.6,-18.0,72.0,39.6,7.2,-32.4,43.2,118.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 21:30:00,-5306.4,205.2,208.8,39.6,183.6,284.4,316.8,176.4,133.2,129.6,...,604.8,-259.2,32.4,115.2,169.2,100.8,10.8,-100.8,122.4,309.6
2017-12-31 22:00:00,-4910.4,208.8,183.6,39.6,154.8,252.0,295.2,165.6,133.2,133.2,...,565.2,-241.2,21.6,100.8,154.8,86.4,18.0,-75.6,100.8,288.0
2017-12-31 22:30:00,-4683.6,187.2,183.6,36.0,147.6,230.4,320.4,147.6,118.8,126.0,...,529.2,-226.8,21.6,100.8,147.6,86.4,14.4,-86.4,108.0,266.4


In [5]:
reframed_demand_node16

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var9(t-1),var10(t-1),...,var88(t-1),var89(t-1),var90(t-1),var91(t-1),var92(t-1),var93(t-1),var94(t-1),var95(t-1),var96(t-1),var17(t)
0,-3348.000000,154.800003,169.199997,18.000000,118.800003,169.199997,234.000000,86.400002,82.800003,104.400002,...,-205.199997,-75.599998,-18.000000,118.800003,68.400002,7.2,-54.000000,75.599998,212.399994,144.000000
1,-2959.199951,136.800003,147.600006,18.000000,108.000000,144.000000,237.600006,79.199997,75.599998,90.000000,...,-187.199997,-72.000000,-21.600000,108.000000,61.200001,3.6,-54.000000,72.000000,183.600006,111.599998
2,-2692.800049,129.600006,126.000000,14.400000,104.400002,133.199997,205.199997,72.000000,72.000000,90.000000,...,-176.399994,-64.800003,-18.000000,93.599998,50.400002,3.6,-39.599998,54.000000,151.199997,108.000000
3,-2379.600098,104.400002,100.800003,10.800000,86.400002,122.400002,180.000000,68.400002,57.599998,82.800003,...,-158.399994,-54.000000,-10.800000,79.199997,46.799999,7.2,-32.400002,50.400002,140.399994,90.000000
4,-2228.399902,104.400002,104.400002,10.800000,79.199997,122.400002,183.600006,68.400002,54.000000,75.599998,...,-144.000000,-57.599998,-18.000000,72.000000,39.599998,7.2,-32.400002,43.200001,118.800003,90.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87595,-5306.399902,205.199997,208.800003,39.599998,183.600006,284.399994,316.799988,176.399994,133.199997,129.600006,...,-259.200012,32.400002,115.199997,169.199997,100.800003,10.8,-100.800003,122.400002,309.600006,252.000000
87596,-4910.399902,208.800003,183.600006,39.599998,154.800003,252.000000,295.200012,165.600006,133.199997,133.199997,...,-241.199997,21.600000,100.800003,154.800003,86.400002,18.0,-75.599998,100.800003,288.000000,230.399994
87597,-4683.600098,187.199997,183.600006,36.000000,147.600006,230.399994,320.399994,147.600006,118.800003,126.000000,...,-226.800003,21.600000,100.800003,147.600006,86.400002,14.4,-86.400002,108.000000,266.399994,219.600006
87598,-4262.399902,180.000000,165.600006,32.400002,136.800003,223.199997,241.199997,147.600006,115.199997,111.599998,...,-208.800003,21.600000,93.599998,133.199997,79.199997,14.4,-75.599998,97.199997,255.600006,208.800003


In [6]:
df['node_16_demand']

Timestamp
2017-01-01 00:00:00    144.0
2017-01-01 00:30:00    111.6
2017-01-01 01:00:00    108.0
2017-01-01 01:30:00     90.0
2017-01-01 02:00:00     90.0
                       ...  
2017-12-31 21:30:00    252.0
2017-12-31 22:00:00    230.4
2017-12-31 22:30:00    219.6
2017-12-31 23:00:00    208.8
2017-12-31 23:30:00    169.2
Name: node_16_demand, Length: 87600, dtype: float64

In [4]:
values = df.values
values = values.astype('float32')
reframed_demand_node16 = series_to_supervised(values, 1, 1, vars_to_predict=target_variable_demand_node16)

In [3]:
# convert series to supervised learning
def series_to_supervised(data, n_in, n_out, vars_to_predict=None):
    n_vars = data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i-1))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(n_out):
        cols.append(df[vars_to_predict].shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in vars_to_predict]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in vars_to_predict]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    return agg

# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))

# node 1 indexes
target_variable_demand_node1 = [1]
target_variable_pressures_node1 = [33]
target_variable_flows_node1 = [64]
#node 16 indexes
target_variable_demand_node16 = [16]
target_variable_pressures_node16 = [48]
target_variable_flows_node16 = [79]
#node 29 indexes
target_variable_demand_node29 = [29]
target_variable_pressures_node29 = [61]
target_variable_flows_node29 = [92]

In [4]:
MAE_average = []
maximum_MAE = []
sensors_covered = []
n_columns_to_drop = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75]
for n in n_columns_to_drop:
    MAE = []
    for run in range(1,11):
        values = df.values
        values = values.astype('float32')
        reframed_demand_node16 = series_to_supervised(values, 1, 1, vars_to_predict=target_variable_demand_node16)
        reframed_demand_node16 = reframed_demand_node16.drop(reframed_demand_node16.columns[16], axis=1)
        columns = []
        columns_to_drop = random.sample(list(reframed_demand_node16.columns), n)
        # Drop the selected columns from the dataframe
        if reframed_demand_node16.columns[-1] in columns_to_drop:
            columns_to_drop.remove(reframed_demand_node16.columns[-1])
        reframed_demand_node16.drop(columns_to_drop, axis=1, inplace=True)
        columns.append(columns_to_drop)
        print(columns)
        print('starting df shape:', reframed_demand_node16.shape)
        print(f'This is training run {run} and has the shape: ', reframed_demand_node16.shape)
        print(f'The value of dropped columns is {n}')

        # split into train and test sets
        values = reframed_demand_node16.values
        scaled = scaler.fit_transform(values)
        train, test = train_test_split(scaled, test_size=0.2, random_state=0)

        # split into input and outputs
        train_X, train_y = train[:, :-1], train[:, -1]
        test_X, test_y = test[:, :-1], test[:, -1]

        #define model
        model = xgb.XGBRegressor(
            objective='reg:squarederror',
            n_estimators=100,
            max_depth=6,
            learning_rate=0.1,
            subsample=0.9,
            colsample_bytree=0.8,
            random_state=42
        )

        # fit the model on the training data
        model.fit(train_X, train_y)

        # make predictions on the test data
        y_pred = model.predict(test_X)

        test_y = test_y.reshape((len(test_y), 1))
        inv_test_y = concatenate((test_y, test_X[:, -reframed_demand_node16.shape[1]:]), axis=1)
        test_y_inverse = scaler.inverse_transform(inv_test_y)
        test_y_inverse = test_y_inverse[:,0]

        y_pred = y_pred.reshape((len(y_pred), 1))
        inv_y_pred = concatenate((y_pred, test_X[:, -reframed_demand_node16.shape[1]:]), axis=1)
        y_pred_inverse = scaler.inverse_transform(inv_y_pred)
        y_pred_inverse = y_pred_inverse[:,0]

        # evaluate scaled errors
        mae_inverse = mean_absolute_error(test_y_inverse, y_pred_inverse)
        MAE.append(mae_inverse)
        print(f'Run {run} Mean Absolute Error:', mae_inverse)
    percentage_list = round(reframed_demand_node16.shape[1] / 96 * 100)
    sensors_covered.append(percentage_list)
    MAE_mean = mean(MAE)
    max_MAE = max(MAE)
    maximum_MAE.append(max_MAE)
    MAE_average.append(MAE_mean)
print('Length MAE_average: ', len(MAE_average))
print('Length sensors_covered: ', len(sensors_covered))

[[]]
starting df shape: (87600, 96)
This is training run 1 and has the shape:  (87600, 96)
The value of dropped columns is 0


/Users/alexellard/opt/anaconda3/lib/python3.7/site-packages/xgboost/data.py:96: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


Run 1 Mean Absolute Error: 45.364094
[[]]
starting df shape: (87600, 96)
This is training run 2 and has the shape:  (87600, 96)
The value of dropped columns is 0


/Users/alexellard/opt/anaconda3/lib/python3.7/site-packages/xgboost/data.py:96: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


Run 2 Mean Absolute Error: 45.364094
[[]]
starting df shape: (87600, 96)
This is training run 3 and has the shape:  (87600, 96)
The value of dropped columns is 0


...

Run 8 Mean Absolute Error: 10.32411
[['var72(t-1)', 'var80(t-1)', 'var49(t-1)', 'var68(t-1)', 'var14(t-1)', 'var62(t-1)', 'var74(t-1)', 'var83(t-1)', 'var24(t-1)', 'var35(t-1)', 'var52(t-1)', 'var66(t-1)', 'var20(t-1)', 'var87(t-1)', 'var45(t-1)', 'var86(t-1)', 'var88(t-1)', 'var63(t-1)', 'var75(t-1)', 'var50(t-1)', 'var1(t-1)', 'var55(t-1)', 'var85(t-1)', 'var39(t-1)', 'var40(t-1)', 'var6(t-1)', 'var61(t-1)', 'var51(t-1)', 'var5(t-1)', 'var81(t-1)', 'var37(t-1)', 'var3(t-1)', 'var71(t-1)', 'var70(t-1)', 'var53(t-1)', 'var42(t-1)', 'var23(t-1)', 'var69(t-1)', 'var2(t-1)', 'var91(t-1)', 'var36(t-1)', 'var73(t-1)', 'var8(t-1)', 'var43(t-1)', 'var4(t-1)', 'var10(t-1)', 'var27(t-1)', 'var33(t-1)', 'var41(t-1)', 'var92(t-1)', 'var67(t-1)', 'var26(t-1)', 'var64(t-1)', 'var77(t-1)', 'var15(t-1)', 'var18(t-1)', 'var60(t-1)', 'var7(t-1)', 'var93(t-1)', 'var13(t-1)', 'var32(t-1)', 'var11(t-1)', 'var78(t-1)', 'var19(t-1)', 'var84(t-1)', 'var76(t-1)', 'var46(t-1)', 'var47(t-1)', 'var44(t-1)', 'var

/Users/alexellard/opt/anaconda3/lib/python3.7/site-packages/xgboost/data.py:96: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


Run 9 Mean Absolute Error: 5.3948236
[['var40(t-1)', 'var2(t-1)', 'var13(t-1)', 'var30(t-1)', 'var95(t-1)', 'var66(t-1)', 'var15(t-1)', 'var79(t-1)', 'var1(t-1)', 'var35(t-1)', 'var62(t-1)', 'var75(t-1)', 'var12(t-1)', 'var29(t-1)', 'var4(t-1)', 'var25(t-1)', 'var90(t-1)', 'var27(t-1)', 'var91(t-1)', 'var34(t-1)', 'var84(t-1)', 'var80(t-1)', 'var33(t-1)', 'var39(t-1)', 'var70(t-1)', 'var20(t-1)', 'var22(t-1)', 'var41(t-1)', 'var76(t-1)', 'var50(t-1)', 'var52(t-1)', 'var82(t-1)', 'var87(t-1)', 'var37(t-1)', 'var92(t-1)', 'var8(t-1)', 'var59(t-1)', 'var93(t-1)', 'var81(t-1)', 'var19(t-1)', 'var44(t-1)', 'var42(t-1)', 'var63(t-1)', 'var65(t-1)', 'var9(t-1)', 'var18(t-1)', 'var55(t-1)', 'var69(t-1)', 'var38(t-1)', 'var64(t-1)', 'var78(t-1)', 'var36(t-1)', 'var88(t-1)', 'var24(t-1)', 'var48(t-1)', 'var73(t-1)', 'var32(t-1)', 'var23(t-1)', 'var10(t-1)', 'var45(t-1)', 'var57(t-1)', 'var49(t-1)', 'var43(t-1)', 'var28(t-1)', 'var56(t-1)', 'var89(t-1)', 'var6(t-1)', 'var77(t-1)', 'var61(t-1)', '

/Users/alexellard/opt/anaconda3/lib/python3.7/site-packages/xgboost/data.py:96: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


Run 10 Mean Absolute Error: 10.71742
Length MAE_average:  16
Length sensors_covered:  16
